In [1]:
import os
PATH = './camara'
docs = []
files = os.listdir(PATH)
for f in files:
    with open(PATH + '/' + f, 'r', encoding='utf-8') as file:
        text = ' '.join(file.read().split('-')[2:])
        docs.append(text)

In [2]:
PATH = './gov'
files = os.listdir(PATH)
for f in files:
    with open(PATH + '/' + f, 'r', encoding='utf-8') as file:
        docs.append(file.read())

In [3]:
with open('data.txt', 'w', encoding='utf-8') as f:
    text = ''.join(docs)
    f.write(text)

In [4]:
import nltk
# nltk.download('punkt')
processed_docs = []
for doc in docs:
    sents = nltk.sent_tokenize(doc)
    for sent in sents:
        if sent[0] not in ['(', ')'] and 'etc' not in sent.split(' '):
            processed_docs.append(sent)
print(processed_docs[-2])
print(len(processed_docs))

E Brasil acima de tudo e Deus acima de todos.
36942


In [5]:
from transformers import AutoModel, AutoTokenizer, AutoModelWithLMHead
import torch
bert_model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
NUM_WORDS = 24766
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer.padding_side = 'left'

In [6]:
sequences = []
k=0
for line in processed_docs:
    words = line.strip().split(' ')
    for i in range(2, len(words)):
        n_gram_sequence = words[:i+1]
        sequences.append(n_gram_sequence)

In [7]:
processed_docs[0]

' Obrigado pela oportunidade.'

In [8]:
token = tokenizer.tokenize(processed_docs[0])
# type(tokenizer)
tokenizer.encode(token, add_special_tokens=False, 
                max_length=15, pad_to_max_length=True)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 9172, 6943, 243, 412, 5940, 119]

In [9]:
tokenizer.encode(processed_docs[0], add_special_tokens=False, 
                max_length=15, pad_to_max_length=True)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 9172, 6943, 243, 412, 5940, 119]

In [10]:
teste = tokenizer.encode(" ".join(sequences[0]), add_special_tokens=True, 
                max_length=15, pad_to_max_length=True)

In [11]:
tokens = tokenizer.convert_ids_to_tokens(teste)
tokenizer.convert_tokens_to_string(tokens)

'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [CLS] Obrigado pela oportunidade . [SEP]'

In [12]:
from tqdm import tqdm

input_ids = []
for i in tqdm(range(len(sequences[0:5000]))):
    tokenized =  tokenizer.encode(" ".join(sequences[i]), add_special_tokens=False, 
                     max_length=15, pad_to_max_length=True)
    input_ids.append(tokenized)
    

100%|████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:01<00:00, 4444.19it/s]


In [13]:
# import pickle
# filename = 'input_ids'
# outfile = open(filename,'wb')
# pickle.dump(input_ids, outfile)


In [14]:
# import numpy as np
# np_a = np.array(a)
# X = np_a[:, :-1]
# labels = np_a[:, -1]
# # y = to_categorical(labels, num_classes=NUM_WORDS)
# y = labels

In [15]:
emb = bert_model(torch.tensor(input_ids))[0]

In [16]:
emb.shape

torch.Size([5000, 15, 768])

In [17]:
print(len(input_ids))

5000


In [18]:
emb.shape
X = emb[:, :-1, :]
y = [_id[-1] for _id in input_ids]

In [19]:
# import numpy as np
# X = np.array(X.detach())
# # y = np.array(y.detach())

# X.astype('float32')
# # y.astype('float32')
# print(X.shape)
# print(y.shape)


In [20]:
import numpy as np
X = np.array(X.detach())
y =  np.array(y)
X.astype('float32')

array([[[ 7.39705190e-03, -3.77377898e-01,  4.48862344e-01, ...,
         -4.00825560e-01,  8.34153518e-02, -6.66779280e-01],
        [-1.77535057e-01, -7.61862546e-02,  7.08285749e-01, ...,
          2.96936482e-01,  1.50145531e-01, -5.83333969e-01],
        [-1.41354382e-01, -5.34604453e-02,  7.58167624e-01, ...,
          2.76368648e-01,  1.47458345e-01, -6.04393005e-01],
        ...,
        [ 1.27789289e-01, -4.27269280e-01,  4.40148592e-01, ...,
          3.84050548e-01, -2.96988755e-01, -4.39451337e-01],
        [-4.04620111e-01, -3.59551728e-01,  2.18491361e-01, ...,
         -3.20315897e-01,  6.37672007e-01,  1.97674856e-01],
        [ 3.07935297e-01,  4.09880877e-02, -3.72139484e-01, ...,
          1.35786533e-01,  6.93073645e-02, -3.47974181e-01]],

       [[ 1.52099192e-01, -3.16811800e-01,  6.37885332e-01, ...,
         -4.88253981e-01, -2.35823125e-01, -6.55695319e-01],
        [-2.53929049e-01, -9.21892375e-03,  7.12694585e-01, ...,
          2.75860250e-01, -4.09389585e

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.9)

In [29]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, GRU, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

model = Sequential()
model.add(Input(shape=(14, 768)))
model.add(Bidirectional(LSTM(128, dtype='float')))
model.add(Dropout(0.2))
# model.add(LSTM(128, dtype='float'))
# model.add(Dropout(0.2))
model.add(Dense(24766, activation='softmax', dtype='float'))

optimizer = Adam(lr=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 256)               918528    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 24766)             6364862   
Total params: 7,283,390
Trainable params: 7,283,390
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
history = model.fit(X_train, y_train, epochs=30, batch_size=4096, 
                    validation_data=(X_valid, y_valid))

# model.train_on_batch(X_train, y_train)

Train on 4500 samples, validate on 500 samples
Epoch 1/30
4500/4500 [==============================] - 3s 618us/sample - loss: 10.0572 - accuracy: 0.0111 - val_loss: 7.6031 - val_accuracy: 0.0780
Epoch 2/30
4500/4500 [==============================] - 0s 54us/sample - loss: 7.2437 - accuracy: 0.0869 - val_loss: 6.0108 - val_accuracy: 0.0780
Epoch 3/30
4500/4500 [==============================] - 0s 55us/sample - loss: 5.4128 - accuracy: 0.0873 - val_loss: 6.3080 - val_accuracy: 0.0780
Epoch 4/30
4500/4500 [==============================] - 0s 55us/sample - loss: 5.4355 - accuracy: 0.0864 - val_loss: 6.2483 - val_accuracy: 0.1040
Epoch 5/30
4500/4500 [==============================] - 0s 55us/sample - loss: 5.1918 - accuracy: 0.1173 - val_loss: 6.0694 - val_accuracy: 0.1380
Epoch 6/30
4500/4500 [==============================] - 0s 55us/sample - loss: 4.8697 - accuracy: 0.1482 - val_loss: 5.9574 - val_accuracy: 0.1760
Epoch 7/30
4500/4500 [==============================] - 0s 57us/sampl

In [46]:
import numpy as np
seed_text = "Dilma"
next_words = 10

# index_to_word = {index: word for word, index in tokenizer.word_index.items()}

T = 0.8

for i in range(next_words):
    tokens = tokenizer.encode(seed_text, add_special_tokens=True, 
                max_length=15, pad_to_max_length=True)
    emb = bert_model(torch.tensor([tokens]))[0]
    X = emb[:, :-1, :]
    X = np.array(X.detach())
    X.astype('float32')
    

    probas = model.predict(X, verbose=0)
    probas = np.array(probas[0][1:])
    probas = probas ** (1.0 / T)
    probas /= np.sum(probas)
    predicted = np.random.choice(range(1,NUM_WORDS), p=probas)
#     print(predicted)
    tokens = tokenizer.convert_ids_to_tokens([predicted])
    print(tokens)
    word = tokenizer.convert_tokens_to_string(tokens)
    print(word)
    seed_text += ' ' + word
# #     predicted = model.predict_classes(token_list, verbose=0)[0]
#     try:
#         while index_to_word[predicted] == seed_text[i-1] or index_to_word[predicted] in ['<unknown>', 'etc']:
#             predicted = np.random.choice(range(1,NUM_WORDS), p=probas)
#     except IndexError:
#         pass
# #     print(f'{list(tokenizer.word_index.keys())[1]}', probas[1]/ np.max(probas))
#     seed_text += " " + (index_to_word[predicted] if predicted != 0 else '')
print(seed_text)

['não']
não
['de']
de
['a']
a
['das']
das
['do']
do
['das']
das
['##emos']
##emos
[';']
;
['de']
de
['se']
se
Dilma não de a das do das ##emos ; de se


O lula fala em uma instituição que porventura apoia a orientação de direitos humanos que seja que na coréia levando em conta as forças armadas que hoje estão lá num partido que tinha um dodói qualquer um dado é qualquer proposta de trabalho que cada vez que o congresso não progredisse apenas no brasil conseguindo um ponto de inflexão e taxou inativos hélio do supremo tribunal federal não está na questão de defesa nacional para que em audiência na reserva antes de declarar de substituir a sua igreja mas que a invalidez vai usar o réu